# Работа с ORM в SQL Alchemy 

<a target="_blank" href="https://colab.research.google.com/github/sozykin/middle_python/blob/main/06/06_orm.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.orm import declarative_base, sessionmaker
from sqlalchemy import Column
from sqlalchemy import String, Integer

## Создание класса с отображением в таблицу базы данных

In [ ]:
# Базовый класс для создание классов, которые будут сохраняться в базе данных
Base = declarative_base()

In [ ]:
# Создаем класс и описываем отображение атрибутов в столбцы таблицы базы данных
class Name(Base):
    # Название таблицы в базе данных
    __tablename__ = 'names'

    # Атрибуты  
    id = Column('id', Integer)
    name = Column('name', String)
    number_of_persons = Column('number_of_persons', Integer)
    global_id = Column('global_id', Integer, primary_key=True)
    year = Column('year', Integer)
    month = Column('month', String)

    # Функция инициализации
    def __init__(self, id, name, number_of_persons, global_id, year, month):
        self.id = id
        self.name = name
        self.number_of_persons = number_of_persons
        self.global_id = global_id
        self.year = year
        self.month = month

    # Текстовое представление объекта
    def __repr__(self):
        return(f"{self.id}, {self.name}, {self.number_of_persons}, {self.global_id}, {self.year}, {self.month}")

In [ ]:
# Создаем соединение с базой данных SQLite
engine = create_engine("sqlite:///test.db", echo=True)

In [ ]:
# Создаем таблицы в базе данных
Base.metadata.create_all(bind=engine)

## Создаем объекты класса и сохраняем их в базе данных

In [ ]:
# Создаем сессию
Sesion = sessionmaker(bind=engine)
session = Sesion()

In [ ]:
# Создаем новый объект класса Name
name1 = Name(3, 'Анна', 190, 37750256, 2015, 'январь')

In [ ]:
# Сохраняем объект в базе данных
session.add(name1)
session.commit()

In [ ]:
# Создаем еще один объект
name2 = Name(4, 'Варвара', 190, 37750257, 2015, 'январь')

In [ ]:
# Сохраняем объект в базе данных
session.add(name2)
session.commit()

## Поиск объектов в базе

In [ ]:
# Извлекаем из базы нужные нам объекты
results = session.query(Name).filter(Name.name == 'Варвара')

In [ ]:
for row in results:
    print(row)

In [ ]:
session.close()

## Современный декларативный подход к описанию отображение атрибутов объекта в поля таблицы

In [ ]:
from sqlalchemy.orm import DeclarativeBase
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import Session

In [ ]:
# Базовый класс для создание классов, которые будут сохраняться в базе данных
class Base(DeclarativeBase):
    pass

In [ ]:
# Создаем класс и описываем отображение атрибутов в столбцы таблицы базы данных
class Name2(Base):
    # Название таблицы в базе данных
    __tablename__ = 'names2'

    # Атрибуты  
    id: Mapped[int] 
    name: Mapped[str]
    number_of_persons: Mapped[str] 
    global_id: Mapped[int] = mapped_column(primary_key=True)
    year: Mapped[int]
    month: Mapped[str]

    # Текстовое представление объекта
    def __repr__(self):
        return(f"{self.id}, {self.name}, {self.number_of_persons}, {self.global_id}, {self.year}, {self.month}")

In [ ]:
# Создаем таблицу
Base.metadata.create_all(engine)

## Создаем объекты

In [52]:
# Метод __init__ с именованными аргументами был сгенерирован автоматически
new_name = Name2(id=3,
                 name='Анна', 
                 number_of_persons=190, 
                 global_id=37750256,
                 year=2015, 
                 month='январь')

In [53]:
new_name

3, Анна, 190, 37750256, 2015, январь

In [54]:
new_name2 = Name2(id=4,
                 name='Варвара', 
                 number_of_persons=190, 
                 global_id=37750257,
                 year=2015, 
                 month='январь')

## Сохраняем данные в базе

In [55]:
with Session(engine) as session:
    session.add(new_name)
    session.add(new_name2)
    session.commit()

2023-06-18 16:56:32,099 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-18 16:56:32,100 INFO sqlalchemy.engine.Engine INSERT INTO names2 (id, name, number_of_persons, global_id, year, month) VALUES (?, ?, ?, ?, ?, ?)
2023-06-18 16:56:32,100 INFO sqlalchemy.engine.Engine [cached since 1876s ago] [(3, 'Анна', 190, 37750256, 2015, 'январь'), (4, 'Варвара', 190, 37750257, 2015, 'январь')]
2023-06-18 16:56:32,109 INFO sqlalchemy.engine.Engine COMMIT


## Извлекаем данные из базы

In [56]:
from sqlalchemy import select

In [57]:
with Session(engine) as session:
    results = session.execute(select(Name2))
    for row in results:
        print(row)

2023-06-18 16:56:36,378 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-18 16:56:36,379 INFO sqlalchemy.engine.Engine SELECT names2.id, names2.name, names2.number_of_persons, names2.global_id, names2.year, names2.month 
FROM names2
2023-06-18 16:56:36,379 INFO sqlalchemy.engine.Engine [cached since 731.7s ago] ()
(3, Анна, 190, 37750256, 2015, январь,)
(4, Варвара, 190, 37750257, 2015, январь,)
2023-06-18 16:56:36,381 INFO sqlalchemy.engine.Engine ROLLBACK


In [58]:
with Session(engine) as session:
    results = session.execute(select(Name2).where(Name2.name == 'Анна'))
    for row in results:
        print(row)

2023-06-18 16:56:39,819 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-18 16:56:39,820 INFO sqlalchemy.engine.Engine SELECT names2.id, names2.name, names2.number_of_persons, names2.global_id, names2.year, names2.month 
FROM names2 
WHERE names2.name = ?
2023-06-18 16:56:39,821 INFO sqlalchemy.engine.Engine [cached since 616.9s ago] ('Анна',)
(3, Анна, 190, 37750256, 2015, январь,)
2023-06-18 16:56:39,822 INFO sqlalchemy.engine.Engine ROLLBACK


## Изменяем данные в базе

In [63]:
session = Session(engine)

In [66]:
name = session.execute(select(Name2).where(Name2.name == 'Анна')).scalar_one()

2023-06-18 16:59:49,906 INFO sqlalchemy.engine.Engine SELECT names2.id, names2.name, names2.number_of_persons, names2.global_id, names2.year, names2.month 
FROM names2 
WHERE names2.name = ?
2023-06-18 16:59:49,907 INFO sqlalchemy.engine.Engine [cached since 807s ago] ('Анна',)


In [67]:
name

3, Анна, 190, 37750256, 2015, январь

In [68]:
name.name = 'Мария'

In [69]:
session.dirty

IdentitySet([3, Мария, 190, 37750256, 2015, январь])

In [70]:
session.commit()

2023-06-18 17:00:52,629 INFO sqlalchemy.engine.Engine UPDATE names2 SET name=? WHERE names2.global_id = ?
2023-06-18 17:00:52,629 INFO sqlalchemy.engine.Engine [generated in 0.00060s] ('Мария', 37750256)
2023-06-18 17:00:52,638 INFO sqlalchemy.engine.Engine COMMIT


In [71]:
session.dirty

IdentitySet([])

In [73]:
session.close()

## Удаляем данные из базы

In [72]:
with Session(engine) as session:
    session.delete(new_name)
    session.delete(new_name2)
    session.commit()

2023-06-18 17:06:58,079 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-18 17:06:58,082 INFO sqlalchemy.engine.Engine SELECT names2.id AS names2_id, names2.number_of_persons AS names2_number_of_persons, names2.global_id AS names2_global_id, names2.year AS names2_year, names2.month AS names2_month 
FROM names2 
WHERE names2.global_id = ?
2023-06-18 17:06:58,082 INFO sqlalchemy.engine.Engine [generated in 0.00064s] (37750256,)
2023-06-18 17:06:58,083 INFO sqlalchemy.engine.Engine SELECT names2.id AS names2_id, names2.name AS names2_name, names2.number_of_persons AS names2_number_of_persons, names2.global_id AS names2_global_id, names2.year AS names2_year, names2.month AS names2_month 
FROM names2 
WHERE names2.global_id = ?
2023-06-18 17:06:58,084 INFO sqlalchemy.engine.Engine [cached since 830.5s ago] (37750257,)
2023-06-18 17:06:58,084 INFO sqlalchemy.engine.Engine DELETE FROM names2 WHERE names2.global_id = ?
2023-06-18 17:06:58,085 INFO sqlalchemy.engine.Engine [cached since 8

## Загрузка данных из JSON а базу с помощью SQL Alchemy

In [74]:
import json

In [75]:
with open("names_f.json", "r") as read_file:
    data = json.load(read_file)

In [76]:
data

[{'ID': 1,
  'Name': 'Мария',
  'NumberOfPersons': 252,
  'global_id': 37750254,
  'Year': 2015,
  'Month': 'январь'},
 {'ID': 2,
  'Name': 'Анастасия',
  'NumberOfPersons': 224,
  'global_id': 37750255,
  'Year': 2015,
  'Month': 'январь'},
 {'ID': 3,
  'Name': 'Анна',
  'NumberOfPersons': 190,
  'global_id': 37750256,
  'Year': 2015,
  'Month': 'январь'},
 {'ID': 4,
  'Name': 'Варвара',
  'NumberOfPersons': 190,
  'global_id': 37750257,
  'Year': 2015,
  'Month': 'январь'},
 {'ID': 5,
  'Name': 'Виктория',
  'NumberOfPersons': 186,
  'global_id': 37750258,
  'Year': 2015,
  'Month': 'январь'},
 {'ID': 483,
  'Name': 'Амелия',
  'NumberOfPersons': 8,
  'global_id': 62367755,
  'Year': 2015,
  'Month': 'май'},
 {'ID': 484,
  'Name': 'Камила',
  'NumberOfPersons': 8,
  'global_id': 62367756,
  'Year': 2015,
  'Month': 'май'},
 {'ID': 485,
  'Name': 'Евангелина',
  'NumberOfPersons': 8,
  'global_id': 62367757,
  'Year': 2015,
  'Month': 'май'},
 {'ID': 486,
  'Name': 'Альбина',
  'Numbe

## Записываем данные в базу

In [78]:
with Session(engine) as session:
    for record in data:
        new_name = Name2(id=record['ID'],
                         name=record['Name'], 
                         number_of_persons=record['NumberOfPersons'], 
                         global_id=record['global_id'],
                         year=record['Year'], 
                         month=record['Month'])
        session.add(new_name)
    session.commit()

2023-06-18 17:12:17,919 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-18 17:12:17,994 INFO sqlalchemy.engine.Engine INSERT INTO names2 (id, name, number_of_persons, global_id, year, month) VALUES (?, ?, ?, ?, ?, ?)
2023-06-18 17:12:17,995 INFO sqlalchemy.engine.Engine [cached since 2822s ago] [(1, 'Мария', 252, 37750254, 2015, 'январь'), (2, 'Анастасия', 224, 37750255, 2015, 'январь'), (3, 'Анна', 190, 37750256, 2015, 'январь'), (4, 'Варвара', 190, 37750257, 2015, 'январь'), (5, 'Виктория', 186, 37750258, 2015, 'январь'), (483, 'Амелия', 8, 62367755, 2015, 'май'), (484, 'Камила', 8, 62367756, 2015, 'май'), (485, 'Евангелина', 8, 62367757, 2015, 'май')  ... displaying 10 of 9870 total bound parameter sets ...  (11095, 'Лидия', 9, 2587775514, 2023, 'Апрель'), (11098, 'Мелисса', 8, 2587775517, 2023, 'Апрель')]
2023-06-18 17:12:18,064 INFO sqlalchemy.engine.Engine COMMIT
